In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


In [ ]:
#make sure to always pip install openai when you open this, idk why but it gets mad at u otherwise

In [ ]:
import openai
import cv2
import base64
import csv
import re
import os
import json
from google.colab import drive

def videoCapture(vid):
    try:
        video = cv2.VideoCapture(vid)
        base64Frames = []
        frame_count = 0

        while video.isOpened():
            success, frame = video.read()
            if not success:
                break

            frame_count += 1

            if frame_count % 10 == 0:
                _, buffer = cv2.imencode(".jpg", frame)
                base64Frames.append(base64.b64encode(buffer).decode("utf-8"))


        video.release()

        response = openai.chat.completions.create(
              model = "gpt-4-vision-preview",
               messages=[
                  {"role": "system", "content": "You are an expert in identifying the sentiment of visual content. Currently working with the FBI, your task is to evaluate each frame of a video to determine if the video is 'toxic' or 'not toxic'. Think about this step by step. Please make sure to end every response stating, 'the prior reasoning is why the video is ________' with the underscores being either 'toxic' or 'not toxic'"},
                  {"role": "user", "content": [{'image' : frame} for frame in base64Frames[0:5]]}
              ]
          )

        result = response

        save_result_incrementally(response.choices[0].message.content)
        print(f'Response: {response.choices[0].message.content}')

        return result


    except Exception as e:
        print(f'Error occurred with video: {vid}, error was: {e}')



drive.mount('/content/drive', force_remount=True)


MessageError: Error: credential propagation was unsuccessful

In [ ]:
# write more code for saving results into csv files


# def save_result_incrementally(fullResponse, response):
#     file_path = '/content/drive/My Drive/instagram-reels-dataset/results/'

#     ogPath  = os.getcwd()

#     os.chdir(file_path)

#     with open('gpt-4-vision-preview-COT-full.csv', 'a', newline='', encoding='utf-8') as file: #full responsne
#         writer = csv.writer(file)
#         writer.writerow([fullResponse])

#     with open('gpt-4-vision-preview-COT.csv', 'a', newline='', encoding='utf-8') as file: #only toxic or not
#         writer = csv.writer(file)
#         writer.writerow([response])

#     os.chdir(ogPath)

def save_result_incrementally(response):
    file_path = '/content/drive/My Drive/instagram-reels-dataset/results/'

    ogPath  = os.getcwd()

    os.chdir(file_path)



    with open('gpt-4-vision-preview-COT.csv', 'a', newline='', encoding='utf-8') as file: #only toxic or not
        writer = csv.writer(file)
        writer.writerow([response])

    os.chdir(ogPath)


def csv_to_json(csv_file_path, json_file_path):
    results = []

    with open(csv_file_path, mode='r') as file:
        csv_reader = csv.reader(file)

        for row in csv_reader:
            result = {
                "responses": row[0],
                "truth": row[1]
            }
            results.append(result)

    with open(json_file_path, 'w') as json_file:
        json.dump(results, json_file, indent=4)

def load_results_from_json(file_path):
    with open(file_path, 'r') as f:
        results = json.load(f)
    return results

base_dir = '/content/drive/My Drive/instagram-reels-dataset'
model_name = "gpt-4-vision-preview"
dataset_name = "toxicity"

# csv_base_filename = f'sc_clustering_{model_name}_{dataset_name}_RUN5'  # Be organized in how you name it
# csv_file_path = os.path.join(base_dir, csv_base_filename + '.csv')

# json_base_filename = f'sc_clustering_{model_name}_{dataset_name}'
# json_file_path = os.path.join(base_dir, json_base_filename + '.json')

In [ ]:
# write code to extract the intended answer from the generation - you might need to few shot prompt GPT to get it to answer in a certain format

# write some code to calculate accuracy



# Store as much as you can. Better to save too much than too little. Memory constraints typically aren't a concern for saving results.

# change the keys to be for what we are storing
        # result = {
        #     "question": question,
        #     "true_answer": true_answer,
        #     "raw_output_answer": raw_output_answer, # Note that it's recommended to also store the raw output answer, in case that you need to change your extraction function later.
        #     "extracted_output_answer": extracted_output_answer,
        #     "numerical_true_answer": numerical_true_answer,
        #     "numerical_output_answer": numerical_output_answer
        # }
        # results.append(result)

        # save_result_incrementally(result, csv_file_path)

In [ ]:
def process_videos(subfolder):
    # Main code function which will iterate through the reels or comments and return its extracted results

      responses = []
      ogPath = "/content/drive/My Drive/instagram-reels-dataset/reels/"


      os.chdir(ogPath + subfolder)

      if not os.path.exists(ogPath + subfolder) or not os.path.isdir(ogPath + subfolder):
            print(f"Subfolder {subfolder} does not exist or is not a directory. Skipping...")

      else:
        files = sorted(os.listdir())

        for file in files:

            result = videoCapture(file)


            responses.append(result)

        os.chdir(ogPath)

        return responses





drive.mount('/content/drive', force_remount=True)
os.chdir("/content/drive/My Drive/instagram-reels-dataset/reels")


Mounted at /content/drive


In [ ]:
responses = []


os.chdir("/content/drive/My Drive/instagram-reels-dataset/reels")

subfolders = sorted(os.listdir())

for subfolder in subfolders:




      result = process_videos(subfolder)

      print(result)

      responses.append(result)

      if len(responses) > 320:
        break

os.chdir("/content/drive/My Drive/instagram-reels-dataset/reels")




Response: The images show a sequence of frames from a video featuring a male news reporter. He is talking about something related to "Authorities in Papua New Guinea," as indicated by the captions. While his facial expressions change slightly across the frames, nothing in the content of these images suggests anything toxic. The reporter is professionally dressed, the background appears to be a news studio, and he is not displaying any inappropriate behavior. Therefore, based on the provided images, the video seems to be a regular news report and does not exhibit any characteristics that would classify it as toxic.

The prior reasoning is why the video is not toxic.


KeyboardInterrupt: 

In [ ]:
def read_results(file_path):
    results = []
    with open(file_path, 'r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        results = [row[0] if row else "" for row in reader]  # Extract only the text from each row
    return results

def evaluate_results(results):
    """
    Evaluate the results to categorize them into 'toxic' or 'not toxic' and calculate statistics.
    """
    # Remove the header or any irrelevant first element
    if results and not results[0].strip():
        results.pop(0)

    total = len(results)
    correct = 0
    incorrect = 0
    mistake = 0

    for text in results:
        if not text.strip():
            mistake += 1
            continue

        # Clean the string by replacing non-alphanumeric characters with a space
        cleaned_str = re.sub(r'\W+', ' ', text).lower()

        # Split into words
        words = cleaned_str.split()

        if words:
            last_word = words[-1]  # Last word in the cleaned list
            second_last_word = words[-2] if len(words) > 1 else ""

            # Check for correct "not toxic" evaluations
            if last_word == 'nottoxic' or (second_last_word == 'not' and last_word == 'toxic'):
                correct += 1
            # Check for incorrect "toxic" evaluations
            elif last_word == 'toxic' and second_last_word != 'not':
                incorrect += 1
            # Everything else is considered a mistake
            else:
                mistake += 1
        else:
            mistake += 1

    if total > 330:
        return 'Error: Total evaluations exceed the number of videos.'

    accuracy = round(correct / total, 2) if total > 0 else 0

    return f"Total Correct: {correct}\nTotal Incorrect: {incorrect}\nMistakes: {mistake}\nAccuracy: {accuracy}"

# Assuming the file is located in the given path
file_path = '/content/drive/My Drive/instagram-reels-dataset/results/gpt-4-vision-preview-COT.csv'

# Read and evaluate the results
results = read_results(file_path)
evaluation = evaluate_results(results)

# Print the evaluation
print(evaluation)

Total Correct: 131
Total Incorrect: 1
Mistakes: 182
Accuracy: 0.42


In [ ]:
#csv writer and append are weird for making files so this code is just to create the files and avoid the issue

file_path = '/content/drive/My Drive/instagram-reels-dataset/results/'

ogPath  = os.getcwd()

os.chdir(file_path)

with open('gpt-4-vision-preview-COT-full.csv', 'w', newline='', encoding='utf-8') as file: #full responsne
        writer = csv.writer(file)
        writer.writerow('')



with open('gpt-4-vision-preview-COT.csv', 'w', newline='', encoding='utf-8') as file: #only toxic or not
        writer = csv.writer(file)
        writer.writerow('')

os.chdir(ogPath)

